# import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('..')

from src.data_fetcher import StockDataFetcher
from src.features import FeatureEngineer

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("Libraries loaded successfully")

# load data

In [ ]:
fetcher = StockDataFetcher('NVDA')
df = fetcher.get_historical_data(period='2y')

print(f"Data shape: {df.shape}")
df.head()

# info

In [ ]:
print("Dataset Information:")
print(f"Total rows: {len(df)}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nMissing values:\n{df.isnull().sum()}")
print(f"\nBasic statistics:")
df.describe()

# price chart

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

ax.plot(df['Date'], df['Close'], linewidth=2, color='#2ecc71', label='Close Price')
ax.fill_between(df['Date'], df['Close'], alpha=0.3, color='#2ecc71')

ax.set_title('NVIDIA Stock Price Over Time', fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Date', fontsize=13)
ax.set_ylabel('Price (USD)', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# volume analysis

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

ax1.plot(df['Date'], df['Close'], linewidth=2, color='#3498db')
ax1.set_title('Stock Price', fontsize=14, fontweight='bold')
ax1.set_ylabel('Price (USD)', fontsize=12)
ax1.grid(True, alpha=0.3)

ax2.bar(df['Date'], df['Volume'], width=1, color='#e74c3c', alpha=0.6)
ax2.set_title('Trading Volume', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Volume', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# daily returns distribution

In [ ]:
df['Daily_Return'] = df['Close'].pct_change() * 100

fig, ax = plt.subplots(figsize=(12, 6))

ax.hist(df['Daily_Return'].dropna(), bins=60, color='#9b59b6', alpha=0.7, edgecolor='black')
ax.axvline(df['Daily_Return'].mean(), color='red', linestyle='--', linewidth=2, 
           label=f'Mean: {df["Daily_Return"].mean():.2f}%')
ax.axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)

ax.set_title('Distribution of Daily Returns', fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Daily Return (%)', fontsize=13)
ax.set_ylabel('Frequency', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean daily return: {df['Daily_Return'].mean():.3f}%")
print(f"Std deviation: {df['Daily_Return'].std():.3f}%")
print(f"Best day: {df['Daily_Return'].max():.2f}%")
print(f"Worst day: {df['Daily_Return'].min():.2f}%")

# price with moving average

In [ ]:
df['SMA_30'] = df['Close'].rolling(window=30).mean()
df['SMA_50'] = df['Close'].rolling(window=50).mean()

fig, ax = plt.subplots(figsize=(15, 7))

ax.plot(df['Date'], df['Close'], linewidth=2, label='Close Price', color='#34495e')
ax.plot(df['Date'], df['SMA_30'], linewidth=2, label='SMA 30', color='#e67e22', linestyle='--')
ax.plot(df['Date'], df['SMA_50'], linewidth=2, label='SMA 50', color='#16a085', linestyle='--')

ax.set_title('Stock Price with Moving Averages', fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Date', fontsize=13)
ax.set_ylabel('Price (USD)', fontsize=13)
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# monthly returns heatmap

In [ ]:
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month

monthly_returns = df.groupby(['Year', 'Month'])['Daily_Return'].sum().reset_index()
pivot_returns = monthly_returns.pivot(index='Month', columns='Year', values='Daily_Return')

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(pivot_returns, annot=True, fmt='.1f', cmap='RdYlGn', center=0, 
            cbar_kws={'label': 'Return (%)'}, linewidths=0.5, ax=ax)

ax.set_title('Monthly Returns Heatmap', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Month', fontsize=12)

plt.tight_layout()
plt.show()